## Contents
<div id="toc-wrapper" style=""><div id="toc" style="max-height: 742px;"><ol class="toc-item"><li><a href="#1.-Data-Exploration-and-Cleaning">1. Data Exploration and Cleaning</a><ol class="toc-item"><li><a href="#1.1-Handling-NAs">1.1 Handling NAs</a></li><li><a href="#1.2-Cleaning-Text-Fields">1.2 Cleaning Text Fields</a></li><li><a href="#1.3-Checking-Uniques-and-Cleaning">1.3 Checking Uniques and Cleaning</a></li><li><a href="#1.4-Fixing-the-Datatypes">1.4 Fixing the Datatypes</a></li><li><a href="#1.5-Analyzing-number_of_products-and-creating-a-SKU-master-data">1.5 Analyzing number_of_products and creating a SKU master data</a></li><li><a href="#1.6-Analyzing-Customer-and-the-Demographics">1.6 Analyzing Customer and the Demographics</a></li><li><a href="#1.7-Country-vs-City">1.7 Country vs City</a></li><li><a href="#1.8-Payments">1.8 Payments</a></li></ol></li><li><a href="#2.-Feature-Engineering">2. Feature Engineering</a></li><li><a href="#3.-Visualizations">3. Visualizations</a><ol class="toc-item"><li><a href="#3.1-Revenue">3.1 Revenue</a></li><li><a href="#3.2-Basket-Size">3.2 Basket Size</a></li><li><a href="#3.3-Products">3.3 Products</a></li><li><a href="#3.4-Frequencies---is_first_order,-country,-payment_type-and-user_gender">3.4 Frequencies - is_first_order, country, payment_type and user_gender</a></li><li><a href="#3.5-Coupons">3.5 Coupons</a></li><li><a href="#3.6-Number-of-Hits-on-Website">3.6 Number of Hits on Website</a></li><li><a href="#3.7-How-is-the-Revenue-affected-by-Days">3.7 How is the Revenue affected by Days</a></li><li><a href="#3.8-Multi-Purchasers">3.8 Multi Purchasers</a></li><li><a href="#3.9-Geographic-Importance">3.9 Geographic Importance</a></li></ol></li><li><a href="#4.-Product-Affinity">4. Product Affinity</a></li><li><a href="#5.-Customer-Segmentation">5. Customer Segmentation</a></li></ol></div></div>

## 1. Data Exploration and Cleaning

In [2]:
pip install folium

     |████████████████████████████████| 109 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 85 kB 9.7 MB/s  eta 0:00:01
     |████████████████████████████████| 133 kB 40.2 MB/s eta 0:00:01
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 1.1.1
    Uninstalling MarkupSafe-1.1.1:
      Successfully uninstalled MarkupSafe-1.1.1
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
cookiecutter 1.7.2 requires Jinja2<3.0.0, but you have jinja2 3.1.4 which is incompatible.
cookiecutter 1.7.2 requires MarkupSafe<2.0.0, but you have markupsafe 3.0.2 which is incompatible.
Note: you may need to restart the ker

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import itertools
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from tqdm.auto import tqdm
from datetime import datetime
from wordcloud import WordCloud

ModuleNotFoundError: No module named 'folium'

In [ ]:
## read the original excel, rename the columns and save it in csv
df = pd.read_csv('MIQ_data_cleaned.csv')

In [ ]:
# df top 5 rows

df.head(5)

### 1.1 Handling NAs

In [ ]:
print(f'Shape :: {df.shape}')

print('NA :: ')
print(df.isna().sum()/df.shape[0]*100) # get percentage

* From the given data we can see that the **birthday** is missing to a varying extent. There can be some cool imputation model that can be built based on the type of product bought. Given the time constraint, it is currently taken out of scope. So, we drop it.

* We drop the column **country province** as of now.

* We will impute the **coupons** with 'na' because it can unearth information that if a particular product is sku is only of promo type.

* We drop the rows where **gender** and the **city** are missing.

In [ ]:
# define drop columns
to_drop_cols = ['user_birthday', 'country_province']
to_drop_rows = ['user_gender', 'city']
to_fillna = {'order_coupon_code': 'na'}

# drop the columns/rows as discussed
df = df.drop(to_drop_cols, axis=1)
df = df.dropna(subset=to_drop_rows)
df = df.fillna(to_fillna)

With the NA removal, we have lost $(49999-48491)/49999*100 = 3.02$% of the data which is acceptable.

### 1.2 Cleaning Text Fields

Let's do a quick check on the city and the country.

In [ ]:
# get the unique city name with original casing and lower casing
df.city.nunique(), df.city.str.lower().nunique(), df.city.str.strip().str.lower().nunique()

We can see that there's discrepancy in the city names wrt casing. So, we lower and strip all the fields which can have text.

In [ ]:
# change casing and trim extra spaces
def text_preprocess(s):
    return s.str.strip().str.lower()

# normalize all the text columns
for c in df.select_dtypes('object').columns:
    df[c] = text_preprocess(df[c])

### 1.3 Checking Uniques and Cleaning

Let's see some of the unique value counts and see if they make some sense.

In [ ]:
df.nunique()

Before we discuss any further, let's check the unique values of and also the datatypes.

In [ ]:
# check the unique values in the below columns
check_unq_val_cols = ['is_first_order', 'country', 'payment_type', 'user_gender']

for i in check_unq_val_cols:
    print(f'Checking {i} :: {df[i].unique()}')

So, we unearth the hidden **undefined** values present in the data. We need to get rid of them for the future analysis.

In [ ]:
# replace undefined with nan and then drop them
df = df.replace('undefined', np.nan)

print('NA :: ')
print(df.isna().sum()/df.shape[0]*100)

So, we find only 0.3% of the data has noise. Let's get rid of them. 

In [ ]:
df = df.dropna()

* **Number of products as 258** is also a bit confusing as it is highly unlikely that a person is shopping that many.
* Since the **product name** and the **product IDs** have different unique count, then we can infer that there is **no 1:1 mapping between them**, which is a bit confusing.
* Since we have lesser ids than the timestamp, we can infer that we have **repeat-customers**.
* From the above uniques, we can see that the number of IPs and user IDs are different. Although it can be the case because no IPs are static nowadays, we can just check and in case find the IPs are varying for the same customer, we **can get rid of IP** too.

### 1.4 Fixing the Datatypes

In [ ]:
df.dtypes

Let's consider changing the datatypes.

In [ ]:
# change the datatypes to the required format
df['timestamp'] = pd.to_datetime(df.timestamp)
df['is_first_order'] = df.is_first_order.astype('int')
df['revenue'] = df.revenue.astype('float')

### 1.5 Analyzing number_of_products and creating a SKU master data

Mostly all except number of products could not be casted to integer because of *invalid literal for int() with base 10: '1,1'*. This makes us rethink the level at which the data is present and decide on it.

In [ ]:
df[['timestamp', 'user_id', 'ip_address', 'user_gender','city', 'country', 'product_id', 'number_of_products']].head(10)

So, the first thing to note here is row 6 has 2 transactions (NI537AA69RPY and NI537AA97PBM). So, essentially **each transaction shows a basket** which we may keep like this or melt it. Let's see how different are the number_of_products are.

In [ ]:
df.number_of_products.unique()[:5]

In [ ]:
df[df.number_of_products == '1,2,1'][['timestamp', 'user_id', 'ip_address', 'user_gender','city', 'country', 'product_id', 'number_of_products']].head(5)

In [ ]:
# create lists from the text of product id, names and number bought
df['product_id'] = df.product_id.apply(lambda x: x.split(','))
df['product_name'] = df.product_name.apply(lambda x: x.split(','))
df['number_of_products'] = df.number_of_products.apply(lambda x: list(map(int, x.split(','))))

# do some basic feature engineering
# create columns such as id count, names count and quantity count
df['product_id_count'] = df.product_id.apply(lambda x: len(x))
df['product_name_count'] = df.product_name.apply(lambda x: len(x))
df['total_products'] = df.number_of_products.apply(lambda x: sum(x))

In [ ]:
print(df[df.product_id_count != df.product_name_count][['product_id', 'product_id_count', 'product_name', 'product_name_count']].shape)
df[df.product_id_count != df.product_name_count][['product_id', 'product_id_count', 'product_name', 'product_name_count']].head()

We can infer that around 264 transactions have noise and the number of product names does not map with the number of product ids. For cleaning the data, we need to get rid of them too.

In [ ]:
# get rid if the transactions were the counts of id and names are not same
df = df[df.product_id_count == df.product_name_count]

Next, we ought to check if there is a 1:1 mapping between the product ids and names.

In [ ]:
# create a flat list of the product names and ids
p_id = [j for i in df.product_id for j in i]
p_name = [j for i in df.product_name for j in i]

# create a df with the id and names
sku_master = pd.DataFrame(columns = ['id', 'name'])
sku_master['id'] = p_id
sku_master['name'] = p_name
sku_master = sku_master.drop_duplicates()

# find the ids which has more than 1 name
t = sku_master.groupby('id').count()
print(f'{t[t.name>1].shape[0]/t.shape[0]*100:.2f}% of product ids have more than 1 description')
print(t[t.name>1].head())

# normalize the ids with more than one name with the name haaving maximum length
sku_master = pd.DataFrame(sku_master.groupby('id').name.apply(list))
sku_master = sku_master.name.apply(lambda x: max(x, key=len))

# reset the product names in the original df
df['product_name'] = [[sku_master[j] for j in i] for i in df.product_id]

We normalized all the product names by taking the longest possible available description for the same item.
With the removals, we have lost $(49999−48214)/49999∗100=3.57$ % of the data till now.



So, in these examples, we can confirm that number of products actually shows the quantity.

However, in the initial head, there seems to be one more observation. `The user_id==0` is making purchases at a gap of 2 minutes from different countries and also has different gender.

This makes us question the following things.
* Is the user_id consistent with the demographics?
* Is the country/city is based on IP?

### 1.6 Analyzing Customer and the Demographics

In [ ]:
demographics_cols = ['user_gender', 'country']
t = df.groupby('user_id').agg(dict(zip(demographics_cols, ['nunique']*len(demographics_cols))))

# get metrics where user has more than 1 gender and country
print(f'Unique Customers :: {t.shape[0]}')
print(f'Customers who have more than 1 demographics info:: {[(c, t[t[c]>1].shape[0]) for c in demographics_cols]}')
print(f'Transaction which comprises of these users :: {df[df.user_id.isin(list(set([j for i in [t[t[c]>1].index for c in demographics_cols] for j in i])))].shape[0]/df.shape[0]*100:.2f}%')

Since it is only around 1.2% of the transactions, we get rid of them.

In [ ]:
df = df[~df.user_id.isin(list(set([j for i in [t[t[c]>1].index for c in demographics_cols] for j in i])))]

### 1.7 Country vs City
How can a city have 2 countries?

In [ ]:
# get count of country for each city
t = df.groupby('city').agg({'country': 'nunique'})
t = t[t.country>1]
t.head()

Now definitely this can be true, that is a place with same name co-exist in two countries. But we have been checking on Google Maps and some of them exist only in a Australia and not New Zealand.

What we come up with is, normalizing by mode.

In [ ]:
# normalize the country which has the highest occurence for a city
def normalize_country(city):
    country = df[df.city == city]['country'].value_counts().index[0]
    df.loc[df.city == city, 'country'] = country
    
# normalize the counties
for city in t.index:
    normalize_country(city)

### 1.8 Payments

There is something called 'no-payment'. What does it mean? Let's check the revenue earned from them.

In [ ]:
df[df.payment_type == 'nopayment']['revenue'].value_counts()

We can see that these transactions contributed nothing to revenue. Let's drop them too.

In [ ]:
# drop cases where there is nopayment
df = df[df.payment_type != 'nopayment']
df = df.drop(['ip_address', 'product_name_count'], axis=1)

We have lost $(49999−46905)/49999∗100=6.2$ % data because of all the treatments.

Hopefully, the data is clean now. Also, we already identified some of the columns for whom we can see the distributions.

## 2. Feature Engineering

In [ ]:
# create some new columns from timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df.timestamp.apply(lambda x: str(x).split()[0])
df['hour'] = df.timestamp.apply(lambda x: str(x).split()[1].split(':')[0])
df['weekday'] = df.timestamp.apply(lambda x: x.weekday()) # Monday is 0, Sunday == 6
df['week_no'] = df.timestamp.apply(lambda x: x.week)

## 3. Visualizations

### 3.1 Revenue

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = [j for i in fig.subplots(2,2) for j in i]

df.revenue.plot(kind='hist', color='gray', ax=ax[0]);
df.revenue.plot(kind='box', color='gray', ax=ax[1]);
df[df.revenue<200].revenue.plot(kind='hist', color='gray', ax=ax[2]);
df[df.revenue<200].revenue.plot(kind='box', color='gray', ax=ax[3]);

ax[0].set_xlabel('revenue')
ax[0].set_title('Frequency Distribution of Revenue')

ax[1].set_title('Box Plot of Revenue')

ax[2].set_xlabel('revenue')
ax[2].set_title('Frequency Distribution of Revenue (after clipping outliers)')

ax[3].set_title('Box Plot of Revenue (after clipping outliers)')

plt.tight_layout()

This gives rise to the question - Is the revenue related with the number of products bought (basket size)?

### 3.2 Basket Size

In [ ]:
fig = plt.figure(figsize=(7,4))
ax = fig.subplots()

df.total_products.value_counts().plot(kind='bar', ax=ax, color='gray')
ax.set_title('Distribution of Basket Size')
ax.set_ylabel('count')
ax.set_xlabel('basket size');

In [ ]:
fig = plt.figure(figsize=(15,4))
ax = [i for i in fig.subplots(1,2)]

sns.scatterplot(x=df.total_products, y=df.revenue, ax=ax[0], color='gray')
sns.violinplot(x=df.total_products, y=df.revenue, ax=ax[1])

ax[0].set_title('Basket Size vs Revenue - Scatter')
ax[0].set_xlabel('basket size')

ax[1].set_title('Basket Size vs Revenue - Violin')
ax[1].set_xlabel('basket size');

In [ ]:
# df.groupby('user_id').timestamp.count().value_counts().plot(kind='bar', color='gray', figsize=(7,4))
# plt.title('Frequency of Purchases by Users')
# plt.xlabel('purchase count');

In [ ]:
print(f'Total Sales :: {df.revenue.sum():.2f}')
print(f'Average Basket Size :: {np.mean(df.total_products):.2f}')
print(f'Average Revenue per transaction :: {np.mean(df.revenue):.2f}')
print(f'Average Revenue per quantity of Product :: {np.mean(df.revenue/df.total_products):.2f}')

### 3.3 Products

In [ ]:
p_name = [j for i in df.product_name for j in i]
p_id = [j for i in df.product_id for j in i]
p_count = [j for i in df.number_of_products for j in i]

t = pd.DataFrame(columns = ['name', 'count'])
t['name'] = p_name
t['count'] = p_count

In [ ]:
print(f'Shape of SKU master :: {sku_master.shape[0]}')
print(f'Shape of SKU master unique SKU names :: {sku_master.drop_duplicates().shape[0]}')
print(f'Shape of unique SKU sold by names :: {len(set(p_name))}')

print(f'Shape of unique SKU sold by ids :: {len(set(p_id))}')
print('It has to be noted that 2 products can have same name, but different ids.')

print(f'Overall SKU sales :: {sum(p_count)} or {df.total_products.sum()}')

In [ ]:
t.groupby('name').sum().sort_values('count', ascending=False)[:25].plot(kind='bar', color='gray', figsize=(7,4))

plt.xlabel('Product Name')
plt.title('Top 25 Products');

In [ ]:
t['count'].value_counts().plot(kind='bar', color='gray', figsize=(7,4))
plt.title('Distribution of times a Product was bought')
plt.xlabel('times a product was bought');

### 3.4 Frequencies - is_first_order, country, payment_type and user_gender

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = [j for i in fig.subplots(2,2) for j in i]

for c, i in enumerate(check_unq_val_cols):
    df[i].value_counts().plot(kind='bar', color='gray', ax=ax[c]);
    ax[c].set_title(i)
    ax[c].set_ylabel('count')
    
plt.tight_layout();

We can see that we have a number of repeat customers. Let's see the % of coupon used among the repeat customers and the new ones.

### 3.5 Coupons

In [ ]:
pd.Series({'coupon': df[df.order_coupon_code != 'na'].shape[0], 
           'no_coupon': df[df.order_coupon_code == 'na'].shape[0]}).plot(kind='bar', color='gray', figsize=(7, 4))

plt.ylabel('count');

In [ ]:
t1 = df['is_first_order'].value_counts()
t2 = df[df.order_coupon_code != 'na']['is_first_order'].value_counts()

(t2/t1).plot(kind='bar', color='gray', figsize=(7, 4))
plt.ylabel('percentage')
plt.title('% of is_first_order when a coupon is applied');

In [ ]:
t1 = df['payment_type'].value_counts()
t2 = df[df.order_coupon_code != 'na']['payment_type'].value_counts()

(t2/t1).plot(kind='bar', color='gray', figsize=(7, 4))
plt.ylabel('percentage')
plt.title('% of payments when a coupon is applied');

* Since we have lesser ids than the timestamp, we can infer that we have **repeat-customers**.
* From the above uniques, we can see that the number of IPs and user IDs are different. Although it can be the case because no IPs are static nowadays, we can just check and in case find the IPs are varying for the same customer, we **can get rid of IP** too.
* Since the **product name** and the **product IDs** have different unique count, then we can infer that there is **no 1:1 mapping between them**, which is a bit confusing.
* Number of products as 258 is also a bit confusing as it is highly unlikely that a person is shopping that many.

### 3.6 Number of Hits on Website

In [ ]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
ax = df.groupby('weekday').agg({'timestamp': 'count'}).plot(figsize=(9, 5), color='gray')

ax.set_xticks(range(7))
ax.set_xticklabels(day_order)

ax.set_ylabel('count')
ax.set_title('Count of Transactions for Each Day');

In [ ]:
ax = df.groupby('weekday').agg({'revenue': 'sum'}).plot(figsize=(9, 5), color='gray')

ax.set_xticks(range(7))
ax.set_xticklabels(day_order)

ax.set_ylabel('sum')
ax.set_title('Sum of Transactions for Each Day');

In [ ]:
t = df.groupby(['week_no', 'weekday']).agg({'timestamp': 'count'}).reset_index().sort_values(['week_no', 'weekday'])

ax = t.timestamp.plot(figsize=(9, 5), color='gray')

ax.set_xticks(range(0, 21))
ax.set_xticklabels([dict(zip(range(7), day_order))[i] for i in t.weekday])

ax.axvline(x=3, color='r', linestyle='--')
ax.axvline(x=9, color='r', linestyle='--')
ax.axvline(x=16, color='r', linestyle='--')

plt.xticks(rotation=90)

ax.set_ylabel('count')
ax.set_title('Count of Transactions for Each Day');

In [ ]:
t = df.groupby(['week_no', 'weekday']).agg({'revenue': 'sum'}).reset_index().sort_values(['week_no', 'weekday'])
t = pd.DataFrame(list(itertools.product(range(39, 43), range(0, 7))), columns=['week_no', 'weekday']).merge(t, how='left')
t = pd.DataFrame(dict(t.groupby('week_no')['revenue'].apply(list)))
t.columns = [f'week_no_{i}' for i in t.columns]

fig = plt.figure(figsize = (9, 5))
ax = fig.add_subplot()
# ax.plot(t.week_no_39, color = "red", alpha = .5, label='week_39')
ax.plot(t.week_no_40, color = "blue", alpha = .5, label='week_40')
ax.plot(t.week_no_41, color = "green", alpha = .5, label='week_41')
ax.plot(t.week_no_42, color = "black", alpha = .5, label='week_42')

ax.set_xticks(range(7))
ax.set_xticklabels(day_order)
plt.legend(loc="upper right")

ax.set_ylabel('sum of revenue')
ax.set_title('Sum of Revenue for Each Weekday across Weeks');

In [ ]:
ax = df.groupby(['hour']).agg({'revenue': 'sum'}).plot(figsize=(9, 5), color='gray')

ax.set_xticks(range(0, 24))
ax.set_xticklabels(['12 am'] + [f'{i} am' for i in range(1, 12)] + ['12 pm'] + [f'{i} pm' for i in range(1, 12)])

plt.xticks(rotation=90)

ax.axvline(x=10.5, color='r', linestyle='--')
ax.axvline(x=18.5, color='r', linestyle='--')

ax.set_ylabel('sum of revenue')
ax.set_title('Sum of Revenue for Each Hour');

In [ ]:
t = df.groupby(['weekday', 'hour']).agg({'revenue': 'count'}).reset_index()
t

fig = plt.figure(figsize = (9, 5))
ax = fig.add_subplot()

ax.plot(t[t.weekday==2].revenue.values, color = "green", alpha = .5, label='Wednesday')
ax.plot(t[t.weekday==5].revenue.values, color = "blue", alpha = .5, label='Saturday')
ax.plot(t[t.weekday==6].revenue.values, color = "black", alpha = .5, label='Sunday')

ax.set_ylim([0, 800])
ax.set_xticks(range(0, 24))
ax.set_xticklabels(['12 am'] + [f'{i} am' for i in range(1, 12)] + ['12 pm'] + [f'{i} pm' for i in range(1, 12)])

ax.axvline(x=10.5, color='r', linestyle='--')
ax.axvline(x=18.5, color='r', linestyle='--');

plt.xticks(rotation=90)

plt.legend(loc="upper right")
ax.set_ylabel('sum of revenue')
ax.set_title('Sum of Revenue for Each Hour across days');

In [ ]:
t = df.groupby(['week_no', 'hour']).agg({'revenue': 'count'}).reset_index()

fig = plt.figure(figsize = (9, 5))
ax = fig.add_subplot()

# ax.plot(t[t.week_no==39].revenue.values, color = "red", alpha = .5, label='week_40')
ax.plot(t[t.week_no==40].revenue.values, color = "blue", alpha = .5, label='week_40')
ax.plot(t[t.week_no==41].revenue.values, color = "green", alpha = .5, label='week_41')
ax.plot(t[t.week_no==42].revenue.values, color = "black", alpha = .5, label='week_42')

ax.set_xticks(range(0, 24))
ax.set_xticklabels(['12 am'] + [f'{i} am' for i in range(1, 12)] + ['12 pm'] + [f'{i} pm' for i in range(1, 12)])

plt.xticks(rotation=90)

ax.axvline(x=10.5, color='r', linestyle='--')
ax.axvline(x=18.5, color='r', linestyle='--')

plt.legend(loc="upper right")
ax.set_ylabel('sum of revenue')
ax.set_title('Sum of Revenue for Each Hour across weeks');

### 3.7 How is the Revenue affected by Days

In [ ]:
t = df.groupby(['week_no', 'weekday']).agg({'revenue': 'mean'}).reset_index().sort_values(['week_no', 'weekday'])

ax = t.revenue.plot(figsize=(9, 5), color='gray')

ax.set_xticks(range(0, 21))
ax.set_xticklabels([dict(zip(range(7), day_order))[i] for i in t.weekday])

ax.axvline(x=0, color='r', linestyle='--')
ax.axvline(x=7, color='r', linestyle='--')
ax.axvline(x=14, color='r', linestyle='--')

plt.xticks(rotation=90)

ax.set_ylabel('mean revenue')
ax.set_title('Mean Revenue for Each Day');

In [ ]:
ax = df.groupby('weekday').agg({'revenue': 'mean'}).plot(figsize=(9, 5), color='gray')

ax.set_xticks(range(7))
ax.set_xticklabels(day_order)

ax.set_ylabel('mean revenue')
ax.set_title('Mean Revenue for Each Weekday');

In [ ]:
t = df.groupby(['week_no', 'weekday']).agg({'revenue': 'mean'}).reset_index().sort_values(['week_no', 'weekday'])
t = pd.DataFrame(list(itertools.product(range(39, 43), range(0, 7))), columns=['week_no', 'weekday']).merge(t, how='left')
t = pd.DataFrame(dict(t.groupby('week_no')['revenue'].apply(list)))
t.columns = [f'week_no_{i}' for i in t.columns]

fig = plt.figure(figsize = (9, 5))
ax = fig.add_subplot()
# ax.plot(t.week_no_39, color = "red", alpha = .5, label='week_39')
ax.plot(t.week_no_40, color = "blue", alpha = .5, label='week_40')
ax.plot(t.week_no_41, color = "green", alpha = .5, label='week_41')
ax.plot(t.week_no_42, color = "black", alpha = .5, label='week_42')

ax.set_ylim([125, 148])
ax.set_xticks(range(7))
ax.set_xticklabels(day_order)
ax.legend(loc="upper right")

ax.set_ylabel('mean revenue')
ax.set_title('Mean Revenue for Each Weekday across Weeks');

In [ ]:
ax = df.groupby(['payment_type']).agg({'revenue': 'mean'}).plot(kind='bar', color='gray', figsize=(7, 4))
ax.set_ylim([0, 170])
ax.legend(loc="upper left")
ax.set_ylabel('mean revenue')
ax.set_title('Mean Revenue across different Payment Types');

### 3.8 Multi Purchasers

In [ ]:
t = df.groupby('user_id').count()[['timestamp']]
print(f'% of customers who are doing multi-purchase :: {t[t.timestamp>1].shape[0]/t.shape[0]*100:.2f}')
print(f'% of customers who are doing single-purchase :: {t[t.timestamp==1].shape[0]/t.shape[0]*100:.2f}')

In [ ]:
mp = df[df.user_id.isin(t[t.timestamp>1].index)]
sp = df[~df.user_id.isin(t[t.timestamp>1].index)]

In [ ]:
ax = mp.hour.astype('int').plot(kind='hist', color='gray', figsize=(7, 4), bins=20)
ax.set_xlabel('hours')
ax.set_title('Histogram of hours at which Purchases are made');

In [ ]:
ax = mp.weekday.astype('int').value_counts().plot(kind='bar', color='gray', figsize=(7, 4))
ax.set_xlabel('days')
ax.set_title('Distribution of Purchases made across Days');

In [ ]:
ax = pd.Series({'multi-purchase': mp.revenue.mean(), 'single purchase': sp.revenue.mean()}).plot(kind='bar', color='gray', figsize=(7, 4))
ax.set_ylim([130, 143])
ax.set_ylabel('mean revenue')
ax.set_title('Mean Revenue of Muti-Purchase vs Single-Purchase');

In [ ]:
t = pd.DataFrame([mp.payment_type.value_counts()/mp.shape[0], sp.payment_type.value_counts()/sp.shape[0]]).T
t.columns = ['multi_purchasers', 'single_purchasers']
ax = t.plot(kind='bar', color=['gray', 'black'], figsize=(7, 4), alpha=0.7)
ax.set_ylim([0, 0.55])

ax.set_ylabel('%purchase')
ax.set_title('% Purchase made via different cards for Sigle and Multi Purchases');

In [ ]:
t = mp[['user_id']].join(mp[['user_id', 'timestamp']].groupby('user_id').diff())
t.timestamp = pd.to_timedelta(t.timestamp).astype(np.int64, errors='ignore').dropna()
t = t.dropna().groupby('user_id').timestamp.apply(list).values

fig = plt.figure(figsize = (7, 4))
ax = fig.add_subplot()
plt.hist([i.days for i in list(map(np.mean, t))], color='gray', bins=20)

ax.set_ylabel('count')
ax.set_xlabel('mean frequency of purchase in days')
ax.set_title('Distribution of Mean Frequency of Purchase (for each user) in days');

In [ ]:
print(f'Average Purchase Frequency: {np.mean([i.days for i in list(map(np.mean, t))]):.2f}')

### 3.9 Geographic Importance

In [ ]:
city = pd.read_csv('worldcities.csv')[['city', 'lat', 'lng', 'population']]
city.city = city.city.str.lower()
t = df.merge(city, on='city')
t = t.groupby(['lat', 'lng']).agg({'revenue': 'mean', 'city': 'max'}).reset_index()

m = folium.Map([-29.043995, 138.264296], zoom_start=4)
for index, row in t.iterrows():
    
    if row['revenue'] >=300:
        fill_color = 'darkred'
    if row['revenue'] >=150:
        fill_color = 'red'
    if row['revenue'] >=100:
        fill_color = 'orange'
    else:
        fill_color = 'darkpurple'
    
    folium.Circle(
          location=[row['lat'], row['lng']],
          tooltip=f"{row['city']} | {row['revenue']}",
          radius=row['revenue']*400,
          color='darkred',
          fill=True,
          fill_color=fill_color,
    ).add_to(m)
m

In [ ]:
city = pd.read_csv('worldcities.csv')[['city', 'lat', 'lng', 'population']]
city.city = city.city.str.lower()
t = df.merge(city, on='city')
t = t.groupby(['lat', 'lng']).agg({'revenue': 'sum', 'city': 'max'}).reset_index()

m = folium.Map([-29.043995, 138.264296], zoom_start=4)
for index, row in t.iterrows():
    
    if row['revenue'] >=5000:
        row['revenue'] = 2000
        fill_color = 'darkred'
    if row['revenue'] >=150:
        fill_color = 'red'
    if row['revenue'] >=100:
        fill_color = 'orange'
    else:
        fill_color = 'darkpurple'
    
    folium.Circle(
          location=[row['lat'], row['lng']],
          tooltip=f"{row['city']} | {row['revenue']}",
          radius=row['revenue']*40,
          color='darkred',
          fill=True,
          fill_color=fill_color,
    ).add_to(m)
m

In [ ]:
ax = df.city.value_counts()[:25].plot(kind='bar', color='gray', figsize=(10, 5))

ax.set_ylabel('count')
ax.set_xlabel('frequency of city')
ax.set_title('Distribution of Frequency of City');

## 4. Product Affinity

In [ ]:
text_1 =  df.product_name.apply(lambda x: ' '.join(x)).str.cat(sep=' ')
wordcloud_1 = WordCloud(background_color='white', collocations=False).generate(text_1)
plt.figure(figsize=(16, 12))
plt.imshow(wordcloud_1, interpolation = "bilinear")
plt.title('Wordcloud for the Product Description', fontsize=15)
plt.axis('off')
plt.show()

These is the wordcloud on the product description. We can see mostly female-related things are sold such as top, lace, sleeves, leather, heel etc. This justifies the gender imbalance in the dataset.

Based on the data, we will run FP Growth to generate the frequent itemsets and then use it for the association rules.

In [ ]:
te = TransactionEncoder()
dataset = df.product_id

dataset = df[df.product_id_count>3].product_id

te_ary = te.fit(dataset).transform(dataset, sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(sparse_df, min_support=0.0007, use_colnames=True, verbose=False)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets2 = frequent_itemsets

rules = association_rules(frequent_itemsets2, metric="confidence", min_threshold=0.6).\
        sort_values(['support', 'confidence'], ascending=False)
rules['antecedents'] = [', '.join([sku_master[j] for j in i]) for i in rules.antecedents]
rules['consequents'] = [', '.join([sku_master[j] for j in i]) for i in rules.consequents]
rules = rules.drop_duplicates(['antecedents', 'consequents'])
rules = rules[rules.antecedents!=rules.consequents][:25]
[f"{i[0]} → {i[1]}" for i in zip(rules.antecedents, rules.consequents) if (i[0] not in i[1])and(i[1] not in i[0])]

## 5. Customer Segmentation

In [ ]:
t = df.groupby('user_id').agg({
    'is_first_order': 'max',
    'user_gender': 'max',
    'country': 'max',
    'revenue': 'mean',
    'total_products': 'max'
})

t = t.replace({'au': 0, 'nz':1, 'male': 0, 'female': 1})
t.head()

In [ ]:
sse = {}
# we will take cluster number from 1 to 50 and perform clustering while noting down the standard squared error
for k in tqdm(range(1, 21)):
    kmeans_elbow = KMeans(n_clusters=k, verbose=False).fit(t)
    # inertia is sum of distances of samples to their closest cluster center
    sse[k] = kmeans_elbow.inertia_

In [ ]:
# plotting the SSE
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(list(sse.keys()), list(sse.values()), color='gray')

# plotting the guidelines to find the optimum number of clusters
ax.axhline(y=93000000, color='r', linestyle="dotted")
ax.axvline(x=4, color='r', linestyle="dotted")

# setting the plot labels and title
plt.xlabel("Number of clusters")
plt.ylabel("SSE")

ax.set_xticks(range(21))
plt.show();

In [ ]:
# scaling the data (recommended for clustering)
me = MinMaxScaler()
t_s = me.fit_transform(t)

In [ ]:
sse = {}
# we will take cluster number from 1 to 50 and perform clustering while noting down the standard squared error
for k in tqdm(range(1, 21)):
    kmeans_elbow = KMeans(n_clusters=k, verbose=False).fit(t_s)
    # inertia is sum of distances of samples to their closest cluster center
    sse[k] = kmeans_elbow.inertia_

In [ ]:
# plotting the SSE
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(list(sse.keys()), list(sse.values()), color='gray')

# plotting the guidelines to find the optimum number of clusters
ax.axhline(y=1200, color='r', linestyle="dotted")
ax.axvline(x=5, color='r', linestyle="dotted")

# setting the plot labels and title
plt.xlabel("Number of clusters")
plt.ylabel("SSE")

ax.set_xticks(range(21))
plt.show();

From the above figures, we can see that the unscaled clustering had a lot of SSE in compared to the scaled one. We proceed with that one and use cluster as 5 because this is a good number after which the SSE almost converges.

In [ ]:
t['cluster'] = KMeans(n_clusters=5, verbose=False, random_state=412).fit_predict(t_s)
t.head()

In [ ]:
ax = t.groupby('cluster').revenue.sum().plot(kind='bar', figsize=(9, 5), color='gray')
ax.set_ylabel('sum of revenue')
ax.set_title('Distribution of Revenue across Clusters');

In [ ]:
ax = t.groupby('cluster').revenue.mean().plot(kind='bar', figsize=(9, 5), color='gray')

ax.set_ylim([100, 145])

ax.set_ylabel('mean of revenue')
ax.set_title('Distribution of Quality of Revenue across Clusters');

**Cluster 0** has the users who have both best quality of revenue and the highest Revenue. <br>
**Cluster 4** has the users who have both worst quality of revenue and the lowest Revenue. <br>

In [ ]:
ax = pd.DataFrame(
    list(
        zip(
            list((t[t.cluster==0].is_first_order.value_counts()/t.is_first_order.value_counts()).fillna(0).values),
            list((t[t.cluster==1].is_first_order.value_counts()/t.is_first_order.value_counts()).fillna(0).values),
            list((t[t.cluster==2].is_first_order.value_counts()/t.is_first_order.value_counts()).fillna(0).values),
            list((t[t.cluster==3].is_first_order.value_counts()/t.is_first_order.value_counts()).fillna(0).values),
            list((t[t.cluster==4].is_first_order.value_counts()/t.is_first_order.value_counts()).fillna(0).values)
        )
    )
).T.plot(kind='bar', figsize=(9, 5), color=['gray', 'black'], alpha=0.6)

l=plt.legend()
l.get_texts()[0].set_text('Old')
l.get_texts()[1].set_text('New')

ax.set_ylabel('% users')
ax.set_xlabel('cluster')
ax.set_title('Distribution of Old and New users across Clusters');

**Cluster 0** has the users who are not first time buyers. <br>
**Cluster 1 and 2** has the users who are first time buyers. <br>

In [ ]:
ax = pd.DataFrame(
    list(
        zip(
            list((t[t.cluster==0].country.value_counts()/t.country.value_counts()).fillna(0).values),
            list((t[t.cluster==1].country.value_counts()/t.country.value_counts()).fillna(0).values),
            list((t[t.cluster==2].country.value_counts()/t.country.value_counts()).fillna(0).values),
            list((t[t.cluster==3].country.value_counts()/t.country.value_counts()).fillna(0).values),
            list((t[t.cluster==4].country.value_counts()/t.country.value_counts()).fillna(0).values)
        )
    )
).T.plot(kind='bar', figsize=(9, 5), color=['gray', 'black'], alpha=0.6)

l=plt.legend()
l.get_texts()[0].set_text('AU')
l.get_texts()[1].set_text('NZ')

ax.set_ylabel('% users')
ax.set_xlabel('cluster')
ax.set_title('Distribution of Users per Country across Clusters');

**Cluster 0** has the users who are mostly from AU. <br>
**Cluster 4** has the users who are from NZ. <br>

In [ ]:
ax = pd.DataFrame(
    list(
        zip(
            list((t[t.cluster==0].user_gender.value_counts()/t.user_gender.value_counts()).fillna(0).values),
            list((t[t.cluster==1].user_gender.value_counts()/t.user_gender.value_counts()).fillna(0).values),
            list((t[t.cluster==2].user_gender.value_counts()/t.user_gender.value_counts()).fillna(0).values),
            list((t[t.cluster==3].user_gender.value_counts()/t.user_gender.value_counts()).fillna(0).values),
            list((t[t.cluster==4].user_gender.value_counts()/t.user_gender.value_counts()).fillna(0).values)
        )
    )
).T.plot(kind='bar', figsize=(9, 5), color=['gray', 'black'], alpha=0.6)

l=plt.legend()
l.get_texts()[0].set_text('male')
l.get_texts()[1].set_text('female')

ax.set_ylabel('% users')
ax.set_xlabel('cluster')
ax.set_title('Distribution of Users per Country across Clusters');

**Cluster 0, 2** has the users who are male. <br>
**Cluster 1 and 3** has the users who are female. <br>

In [ ]:
ax = pd.DataFrame([
    t[t.cluster==0].total_products.mean(),
    t[t.cluster==1].total_products.mean(),
    t[t.cluster==2].total_products.mean(),
    t[t.cluster==3].total_products.mean(),
    t[t.cluster==4].total_products.mean()
]).plot(kind='bar', figsize=(9, 5), color=['gray', 'black'])

ax.set_ylim([1.6, 2.3])

ax.set_ylabel('mean basket size')
ax.set_xlabel('cluster')
ax.set_title('Mean Basket Size across Clusters');

**Cluster 0 and 3** has the users who have a considerable big basket size. <br>
**Cluster 4** has the users who does not have a considerable big basket size.

**cluster 0**: has the **Australian, male, old-users** who have produce **best quality** and the **highest revenue**.<br>
**cluster 1**: has the **first-time, female buyers**.<br>
**cluster 2**: has the **first-time, male buyers.**<br>
**cluster 3**: has the **old female users** who have a **big basket size**.<br>
**cluster 4**: has the **New Zealand users** who produce **least quality revenue** and **neither have a big basket size**.